# 확률적 경사 하강법 (SGD : Stochastic Gradient Descent)
> stochastic : having a random probability distribution or pattern<br>
> 기울기가 0인값을 찾는것 (편미분 )

- 머신러닝에도 많이 사용하지만 실질적으로는 딥러닝에서 많이 사용한다
- 데이터가 추가 되었을 걍우 머신러닝 처리 방법

1. 새로운 데이터로 모델을 추가하여 새롭게 학습하여 처리한다.
2. 추가된 갯수만큼 기존의 데이터를 버리고 사용한다.(과거의 데이터는 필요없다)
3. 기존 모델을 수정하면서 사용한다 (점진적 학습방법)

- 데이터를 추가할 경우에도 새로운 모데을 만들지 않고 기존의 모델을 수정하면서 사용할 수 있는 방법
- 머신러닝 및 딥러닝 알고리즘을 훈련하는 방법 또는 최적화 하는 방법이지 머신러닝이나 딥러닝 모델은 아님
- 확률적 : 랜덤, 경사: 기울기 
- 곡선의 최적점을 찾기 위해 기울기를 기준으로 조금씩 이동하면서 최적적을 찾는 방법

- Epoch: 훈련세트를 다 사용하였을 경우의 단계 <br>
- 미니배치(mini batch)경사 하강법 중 미니배치의 갯수 : hyper parameter, 보통은 2의 배수<br>
- 배치하강법: 샘플을 한번에 몽땅 꺼내면 컴퓨터 메모리나 CPU에 영향을 줄 수 있어서 거의 사용하지 않고 주로 확률적 경사하강법이나 미니배치를 사용한다.

![](../Data/SGD.png)

### 손신할수(비용함수)
- 머신러닝 알고리즘이 에러율을 측정하는 함수
- 손실함수 값이 적은 것이 정확도가 높은 것이다
- 확률적 경사함수를 통해서 손실함수가 낮은 값을 찾는 것이다.
- 분류함수인 경우에 정확도를 사용하는데 이 정확도의 값은 미분할수가 없어서(연속성이 없어) 손실함수로 사용할 수 없다
- 분류함수의 경우에는 로지스틱 손실함수, 이진크로스엔트로피 손실함수르 사용한다.

![](../Data/lost.png)

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
import pandas as pd
fish = pd.read_csv("../Data/fishes.csv")
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [5]:
# feature
# 2차원 배열로 만들기 
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_input[:5]

array([[242.    ,  25.4   ,  30.    ,  11.52  ,   4.02  ],
       [290.    ,  26.3   ,  31.2   ,  12.48  ,   4.3056],
       [340.    ,  26.5   ,  31.1   ,  12.3778,   4.6961],
       [363.    ,  29.    ,  33.5   ,  12.73  ,   4.4555],
       [430.    ,  29.    ,  34.    ,  12.444 ,   5.134 ]])

In [6]:
#target data
fish_target = fish['Species'].to_numpy()
fish_target[:5]

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream'], dtype=object)

In [7]:
# train, test 분리
from sklearn.model_selection import train_test_split  
train_input, test_input,train_target, test_target = \
    train_test_split(fish_input, fish_target,random_state=42)

In [8]:
# 표준화 시키기
# : 경사 하강법을 사용하는 경우에는 샘플의 크기 대문에 경사의 각도가 바귈수 있으므로 각 features의 scale을 표준점수로 변경해야 한다.<br>
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
train_scaled = ss.fit_transform(train_input)
test_scaled = ss.transform(test_input)

---
### 확률적 경사하강법의 분류 알고리즘 적용

In [9]:
from sklearn.linear_model import SGDClassifier

In [10]:
sc = SGDClassifier(
    loss = "log",   
    max_iter= 10 ,
    random_state=42
)
#loss="lost" 손실함수
#max_iter= epoch

In [13]:
sc.fit(train_scaled,train_target)
print("train score : ",sc.score(train_scaled,train_target))
print("test score : ",sc.score(test_scaled,test_target))

train score :  0.773109243697479
test score :  0.775


> scores 가 별로 안 좋고 과소적합이 되어서 <br>
> patial_fit을 사용한다. 기존 w와 b의 계수값(weight, bias)을 유지한 채 update 하는 방법<br>
> 만약에 다시 fit을 사용하면 기존에 학습했던 내용을 모두 버린다.

In [18]:
sc.partial_fit(train_scaled, train_target)
print("train score : ",sc.score(train_scaled,train_target))
print("test score : ",sc.score(test_scaled,test_target))

train score :  0.8571428571428571
test score :  0.9


##### 실행시마다 w,b의 계수가 한번씩 바뀌는 확률적 경사하강법이다.

> 처음의 점수보다 기존에 학습했던 내용이 새로운 학습내용을 수정해서 사용하면 점수가 증가했다.<br>
> 이와 같이 점진적으로 정확도를 높이는 방법이 확률적 경사 하강법이다.<br>
> 처음보다는 점수가 증가하였지만 아직 과소적합니다. 약간의 차이가 나는 과대적합이 제일 좋은 방법이다.